- hidden layer 가 1개인 multi-perceptron을 지원하는 함수 (3)

In [ ]:
# 1. 기존 output layer에 hidden layer 하나만 추가한 가장 간단한 multi-perceptron 지원
# 2. 설정 정보값에 따라 hidden layer 수와 폭을 정하는 multi-perceptron 지원
# 3. 설정 정보 존재 여부에 따라 두 경우 중 하나 선택

mlp.ipynb

In [ ]:
# hidden layer 가 하나인 multi-perceptron NN model의 parameter 를 생성 

def init_model_hidden1():  # init_model 과 비슷하지만, 
    # 두 쌍의 parameter 를 pm_hidden, pm_output에 나눠 저장하고
    # hidden_cnt를 global variable 로 받아 parameter의 크기를 지정 
    global pm_output, pm_hidden, intput_cnt, output_cnt, hidden_cnt
    pm_hidden = alloc_param_pair([input_cnt, hidden_cnt]) # hidden layer
    pm_output = alloc_param_pair([hidden_cnt, output_cnt]) # output layer

def alloc_param_pair(shape): # layer 를 위한 parameter 쌍을 생성 & 초기화 
    weight = np.random.normal(RND_MEAN, RND_STD, shape) # weight 행렬 생성
        # random.normal; shape의 크기를 갖는 무작위 표본 추출 
        # shape; 나중에 높은 차원의 weight tensor 가 필요한 CNN 처리에 유용
    bias = np.zeros(shape[-1]) # bias 벡터 생성 
        # shape 이 2차원을 넘어서는 경우에 대비하여 shape[-1]
    return {'w': weight, 'b': bias}

In [ ]:
# hidden layer 가 하나인 multi-perceptron NN 의 순전파 처리

def forward_neuralnet_hidden1(x):
    global pm_output, pm_hidden # init_model_hidden 에서 만든 parameter 에 접근 
    # hidden layer 의 output ; 비선형 activation function 인 relu 이용 
    hidden = relu(np.matmul(x, pm_hidden['w']) + pm_hidden['b'])
    # 최종 output 
    output = np.matmul(hidden, pm_output['w']) + pm_output['b']
    return output, [x, hidden]

def relu(x):
    return np.maximum(x,0)

In [ ]:
# hidden layer 가 하나인 multi-perceptron NN 의 역전파 처리

def backprop_neuralnet_hidden1(G_output, aux):
    global pm_output, pm_hidden
    x, hidden = aux
    
    g_output_w_out = hidden.transpose()
    G_w_out = np.matmul(g_output_w_out, G_output)
    G_b_out = np.sum(G_output, axis=0)
    # 서로가 서로의 계수가 됨
    g_output_hidden = pm_output['w'].transpose()
    G_hidden = np.matmul(G_output, g_output_hidden)
    
    pm_output['w'] -= LEARNING_RATE * G_w_out
    pm_output['b'] -= LEARNING_RATE * G_b_out
    
    G_hidden = G_hidden * relu_derv(hidden) # relu() 에 대한 역전파 처리 
    
    g_hidden_w_hid = x.transpose()
    G_w_hid = np.matmul(g_hidden_w_hid, G_hidden)
    G_b_hid = np.sum(G_hidden, axis=0)
    
    pm_hidden['w'] -= LEARNING_RATE * G_w_hid
    pm_hidden['b'] -= LEARNING_RATE * G_b_hid

def relu_derv(y): # relu 의 편미분 계산 
    return np.sign(y)

- 임의의 hidden layer 를 갖는 multi-perceptron을 지원하는 함수 (3)

In [ ]:
# 임의의 hidden layer수, 폭을 갖는 model의 paremeter 생성

def init_model_hiddens():
    global pm_output, pm_hiddens, input_cnt, output_cnt, hidden_config
        # hidden_config; hidden layer 의 수, 폭 지정
    pm_hiddens = [] # 생성된 parameter 쌍들 
    # input vector size;  
    prev_cnt = input_cnt # 첫번째 hidden layer일 경우 input_cnt,
    for hidden_cnt in hidden_config: 
        pm_hiddens.append(alloc_param_pair([prev_cnt, hidden_cnt]))
        prev_cnt = hidden_cnt # 아닐 경우 앞 hidden layer의 output vector 크기
        
    # layer 를 위한 parameter 쌍을 생성 & 초기화하는 alloc_param_pair
    pm_output = alloc_param_pair([prev_cnt, output_cnt])   

In [ ]:
# 가변적 multi-perceptron NN 의 순전파 처리 

def forward_neuralnet_hiddens(x):
    global pm_output, pm_hiddens
    # 각 layer를 매개하는 hidden
    hidden = x # 맨 처음 초기화 
    # 역전파 용 보조 정보로 전달하는 hiddens
    hiddens = [x]
    
    for pm_hidden in pm_hiddens:
        # hidden layer의 input 이자 output으로 이용 
        hidden = relu(np.matmul(hidden, pm_hidden['w']) + pm_hidden['b'])
        hiddens.append(hidden)
        
    output = np.matmul(hidden, pm_output['w']) + pm_output['b']
    
    return output, hiddens

In [ ]:
# 가변적 multi-pereptron NN 의 역전파 처리

def backprop_neuralnet_hiddens(G_output, aux):
    global pm_output, pm_hiddens
    hiddens = aux
    # output layer에 대한 역전파 처리 수행
    g_output_w_out = hiddens[-1].transpose()
    G_w_out = np.matmul(g_output_w_out, G_output)
    G_b_out = np.sum(G_output, axis=0)
    
    g_output_hidden = pm_output['w'].transpose() 
    G_hidden = np.matmul(G_output, g_output_hidden)
    # output layer에 대한 역전파 처리 수행
    pm_output['w'] -= LEARNING_RATE * G_w_out
    pm_output['b'] -= LEARNING_RATE * G_b_out
    # hidden layer 수 만큼 역전파 처리
    for n in reversed(range(len(pm_hiddens))):
        G_hidden = G_hidden * relu_derv(hiddens[n+1])
        g_hidden_w_hid = hiddens[n].transpose()
        G_w_hid = np.matmul(g_hidden_w_hid, G_hidden)
        G_b_hid = np.sum(G_hidden, axis=0)
    
        g_hidden_hidden = pm_hiddens[n]['w'].transpose()
        G_hidden = np.matmul(G_hidden, g_hidden_hidden)
        
        pm_hiddens[n]['w'] -= LEARNING_RATE * G_w_hid
        pm_hiddens[n]['b'] -= LEARNING_RATE * G_b_hid

- hideen layer 구성 정보에 알맞은 함수를 호출하는 함수 (3)
    - init_model, forward_neuralnet, backprop_neuralnet 재정의 

In [ ]:
# 설정 정보의 존재여부에 따라 앞서 정의된 두 묶음의 함수를 선택적으로 호출

global hidden_config

def init_model():
    if hidden_config is not None:
        print('은닉 계층 {}개를 갖는 다층 퍼셉트론이 작동되었습니다.'. \
              format(len(hidden_config)))
        init_model_hiddens()
    else:
        print('은닉 계층 하나를 갖는 다층 퍼셉트론이 작동되었습니다.')
        init_model_hidden1()
    
def forward_neuralnet(x):
    if hidden_config is not None:
        return forward_neuralnet_hiddens(x)
    else:
        return forward_neuralnet_hidden1(x)
    
def backprop_neuralnet(G_output, hiddens):
    if hidden_config is not None:
        backprop_neuralnet_hiddens(G_output, hiddens)
    else:
        backprop_neuralnet_hidden1(G_output, hiddens)

In [ ]:
# hidden layer 의 구조 지정

def set_hidden(info):
    global hidden_cnt, hidden_config
     # info 가 정수인지에 따라 
    if isinstance(info, int):
        hidden_cnt = info
        hidden_config = None
    else:
        hidden_config = info